<a href="https://colab.research.google.com/github/CIVE461-Group5/CIVE_461/blob/main/G03_EDWARDS_CLARK_FEHLHAFER_DEMAYO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Your first task is to install Biogeme. Instructions are given on the package webpage: https://biogeme.epfl.ch/install.html

In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
nb_path = '/content/notebooks'
#os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0,nb_path)
#!pip install --target=$nb_path biogeme

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load packages
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.messaging as msg
from biogeme.expressions import Beta, Variable, Derive

# Change me after the first run to save some time
FIRST_RUN = True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
if FIRST_RUN:
    # Load in the trips csv file
    # As written, this will search back one directory level from the current file and in a "Data" folder containing the datasets. The reference will be different unless you define the same directory structure.
    trip_df = pd.read_csv("/content/drive/MyDrive/CIVE 461/CIVE461-Python/CIVE461/CIVE_461-main/Group Project/Data/trips_all.csv")
    # We want to be able to filter by home location (not given in trip data) and include hh and person details so need hh_all.csv/per_all.csv
    hh_df = pd.read_csv("/content/drive/MyDrive/CIVE 461/CIVE461-Python/CIVE461/CIVE_461-main/Group Project/Data/hh_all.csv")
    per_df = pd.read_csv("/content/drive/MyDrive/CIVE 461/CIVE461-Python/CIVE461/CIVE_461-main/Group Project/Data/per_all.csv")
    # We'll need household weights for the zone-based model analysis to generate some approximate zonal statistics
    hh_wgt_df = pd.read_csv("/content/drive/MyDrive/CIVE 461/CIVE461-Python/CIVE461/CIVE_461-main/Group Project/Data/hh_wgt_all.csv")

    # Merge on UID, which is a combination of a household id and a dataset id
    trip_df = trip_df.merge(hh_df.loc[:,["UID","hh_cbsa"]], on="UID")

    trip_df.to_csv("my_hh_trip_data.csv",index=False)
else:
    trip_df = pd.read_csv("my_hh_trip_data.csv")

<ipython-input-143-4ad41d79b5cd>:4: DtypeWarning: Columns (26,28,67,91,122,124,137,138,153,154) have mixed types. Specify dtype option on import or set low_memory=False.
  trip_df = pd.read_csv("/content/drive/MyDrive/CIVE 461/CIVE461-Python/CIVE461/CIVE_461-main/Group Project/Data/trips_all.csv")
<ipython-input-143-4ad41d79b5cd>:6: DtypeWarning: Columns (58) have mixed types. Specify dtype option on import or set low_memory=False.
  hh_df = pd.read_csv("/content/drive/MyDrive/CIVE 461/CIVE461-Python/CIVE461/CIVE_461-main/Group Project/Data/hh_all.csv")
<ipython-input-143-4ad41d79b5cd>:7: DtypeWarning: Columns (2,3,17,53,61,63,85,86,87,115,143,144,145,158,170) have mixed types. Specify dtype option on import or set low_memory=False.
  per_df = pd.read_csv("/content/drive/MyDrive/CIVE 461/CIVE461-Python/CIVE461/CIVE_461-main/Group Project/Data/per_all.csv")


In [ ]:
trip_df.head()

,sampno,perno,tripno,o_locno,locno,strttime,endtime,trvlcmin,distance_mi,trpmiles,...,transit_acct,transit_egrt,transit_waitt,UID,pub_reason,wtperfin5d,wttrdfin5d,wtperfin,wttrdfin,hh_cbsa
0,30000449,1,1,100,102,829,855,26,9.515,9.515,...,8.291029,11.263992,9.576387,30000449.1,NaN,NaN,NaN,NaN,NaN,12260
1,30000449,1,2,102,1000000,905,910,5,0.307,0.307,...,6.256466,7.472391,9.479657,30000449.1,NaN,NaN,NaN,NaN,NaN,12260
2,30000449,1,3,1000000,102,1000,1005,5,0.307,0.307,...,6.256466,7.472391,9.479657,30000449.1,NaN,NaN,NaN,NaN,NaN,12260
3,30000449,1,4,102,1000001,1315,1330,15,1.823,1.823,...,6.256466,7.472391,9.479657,30000449.1,NaN,NaN,NaN,NaN,NaN,12260
4,30000449,1,5,1000001,102,1625,1640,15,1.815,1.815,...,6.256466,7.472391,9.479657,30000449.1,NaN,NaN,NaN,NaN,NaN,12260


# Important columns
Some important columns to make note of when developing your models are given below. Note: we could obtain an estimate for transit cost from GTFS data. We'll ignore transit cost for the purposes of the assignment though.
- auto_tt = auto travel time in minutes
- auto_cost = auto cost in dollars
- transit_tt = transit travel time in minutes
- transit_acct = transit access time in minutes
- transit_egrt = transit egress time in minutes
- transit_waitt = transit wait time in minutes
- trpmiles = trip distance in miles
- ch_mode = {1 = walk, 2 = bike, 3 = auto, 4 = transit}

Biogeme, like most discrete choice software, requires that all data are numeric. We have several columns with non-numeric entries. We'll have to do some processing to prepare the data for use with Biogeme.

In [ ]:
# Filter data for your region
my_trips = trip_df.loc[(trip_df.hh_cbsa=="31080")]

# Home-Based Non-Work Trip Model


In [ ]:
hbnw_filt = ((my_trips.trippurp=="HBO") | (my_trips.trippurp=="HBSHOP") | (my_trips.trippurp=="HBSOCREC"))
# This line will filter for non-work trips. It also removes any column with object datatype.
# If a column appears in the data description but not the dataframe after running this line, you may have to
# do some processing to have it available to you for model estimation. E.g., you can change a string trippurp code to
# an integer code - HBO = 1, etc..
hbnw_trips = my_trips.loc[hbnw_filt].select_dtypes(exclude=['object'])
hbnw_trips.fillna(-1, inplace=True)
# Biogeme requires information on if an alternative is available
# Here, I'm assuming that auto/transit modes are always available. We could exclude auto for those without a license.
# I assume that walk/bike are available for trips with distances less than the maximum reported trip distance with that as the chosen mode
hbnw_trips["walk_av"] = (hbnw_trips.trpmiles   <=  hbnw_trips[hbnw_trips.ch_mode==1].trpmiles.max()).astype(int)
hbnw_trips["bike_av"] = (hbnw_trips.trpmiles   <=  hbnw_trips[hbnw_trips.ch_mode==2].trpmiles.max()).astype(int)
hbnw_trips["auto_av"] = 1
hbnw_trips["transit_av"] = 1

We don't want to include a record if it has a negative value in a critical column. We'll filter the dataframe for those important columns.

In [ ]:
# Update based on if you add more columns to models that potentially contain negative numbers
check_cols = ["auto_tt","auto_cost", "transit_tt", "transit_acct", "transit_egrt", "transit_waitt", "trpmiles", "ch_mode"]
hbnw_trips = hbnw_trips[hbnw_trips.loc[:,check_cols].sum(axis=1)>-1]

In [ ]:
# Need to import data to the Biogeme database structure
database = db.Database("my_hbnw_trips",hbnw_trips)
database.data.head()

# Define Biogeme variables
AUTO_TT = Variable("auto_tt")
AUTO_COST = Variable("auto_cost")
TRANSIT_TT = Variable("transit_tt")
DIST = Variable("trpmiles")
CHOICE = Variable("ch_mode")
WALK_AV = Variable("walk_av")
BIKE_AV = Variable("bike_av")
AUTO_AV = Variable("auto_av")
TRANSIT_AV = Variable("transit_av")
TRANSIT_ACCT = Variable("transit_acct")
TRANSIT_EGRT = Variable("transit_egrt")
TRANSIT_WAITT = Variable("transit_waitt")

We can then define the names and starting values for the model parameters. There are 5 parameters defined for the base Swiss Metro model. Biogeme uses a constrained optimization to reduce the search space. The Beta function is called as follows:
```python
Beta(parameter name AS str, initial value AS int, lower bound AS int, upper bound AS int, fixed AS int)
```
If there are no bounds on the parameter search space, the use may enter: ```None```. A value of ```0``` for the ```fixed``` parameter will allow Biogeme to estimate the parameter. In this case, we have only fixed the ASC for the swiss metro mode, to maintain model indentification.

In [ ]:
ASC_WALK = Beta('ASC_WALK',0,None,None,0)
ASC_BIKE = Beta('ASC_BIKE',0,None,None,0)
ASC_TRANSIT = Beta('ASC_TRANSIT',0,None,None,0)
B_TIME = Beta('B_TIME',0,None,None,0)
B_COST = Beta('B_COST',0,None,None,0)
B_DIST = Beta('B_DIST',0,None,None,0)
B_TIME_BC = Beta('B_TIME_BC',0,None,None,0)
TRANSIT_TT_BC = models.boxcox(TRANSIT_TT, B_TIME_BC)
print(TRANSIT_TT_BC)

{{0:{{0:(((transit_tt ** B_TIME_BC(init=0)) - `1.0`) / B_TIME_BC(init=0)), 1:(((log(transit_tt) + (B_TIME_BC(init=0) * (log(transit_tt) ** `2.0`))) + (((B_TIME_BC(init=0) ** `2.0`) * (log(transit_tt) ** `3.0`)) / `6.0`)) + (((B_TIME_BC(init=0) ** `3.0`) * (log(transit_tt) ** `4.0`)) / `24.0`))}[((B_TIME_BC(init=0) < `1e-05`) * (B_TIME_BC(init=0) > (-`1e-05`)))], 1:`0.0`}[(transit_tt == `0.0`)]


In [ ]:
print (B_TIME)
print (B_COST)
print (B_DIST)

B_TIME(init=0)
B_COST(init=0)
B_DIST(init=0)


The next step is to define the utility function for each alternative mode.

In [ ]:
V1 = ASC_WALK + \
     B_DIST * DIST
V2 = ASC_BIKE + \
     B_DIST * DIST
V3 = B_TIME * AUTO_TT + \
     B_COST * AUTO_COST
V4 = ASC_TRANSIT + \
     B_TIME * TRANSIT_TT_BC

Biogeme has a variety of built-in discrete choice models, which the user can leverage. However, it also allows the user to define their own likelihood function if it is not available in the built-in model library. In this case, we use the built-in ```loglogit``` function to estimate the model.

In [ ]:
# Associate utility functions with the numbering of alternatives
V = {1: V1,
     2: V2,
     3: V3,
     4: V4}

# Associate the availability conditions with the alternatives

av = {1: WALK_AV,
      2: BIKE_AV,
      3: AUTO_AV,
      4: TRANSIT_AV}

logprob_nw = models.loglogit(V,av,CHOICE)
biogeme_nw  = bio.BIOGEME(database,logprob_nw)
biogeme_nw.modelName = "01logit_nw"
results_nw = biogeme_nw.estimate()

The results of the model estimation can then be printed to the terminal.

In [ ]:
# Get the results in a pandas table
# Rob. Std. err are standard errors that are robust to heteroskadastic errors.
# This knowledge is beyond the CIVE461 scope.
pandasResults_nw = results_nw.getEstimatedParameters()
print(pandasResults_nw)

                Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_BIKE    -2.529439      0.780482    -3.240866  1.191674e-03
ASC_TRANSIT -4.077654      0.755180    -5.399577  6.679817e-08
ASC_WALK     0.078729      0.778569     0.101120  9.194552e-01
B_COST      -0.000414      0.002736    -0.151119  8.798818e-01
B_DIST      -1.079365      0.203434    -5.305720  1.122291e-07
B_TIME      -0.006423      0.001379    -4.657238  3.204796e-06
B_TIME_BC   -2.023598      0.196642   -10.290795  0.000000e+00


In [ ]:

asc_bike_estimate = pandasResults_nw.loc['ASC_BIKE', 'Value']

**NOTE:** The above model gives appropriate results for the sample CBSA! This may not be true for your dataset. DO NOT submit results with positive cost, distance, or time parameters.

Biogeme also includes functionalities to compute elasticties for model variables. The elasticity can be calculated directly by the user, but Biogeme also includes functions for calculating derivatives for the user.

We have to redefine the model for it to run in a Jupyter notebook, so there is some repetition in the following code. Note: the parameter definitions have been slightly altered to use the calculated betas. These are stored in a dictionary and the values can be accessed through reference to the appropriate dictionary keys.

In [ ]:
# Get the beta parameters estimated in previous step
betas = results_nw.getBetaValues()

ASC_WALK = Beta('ASC_WALK',betas['ASC_WALK'],None,None,0)
ASC_BIKE = Beta('ASC_BIKE',betas['ASC_BIKE'],None,None,0)
ASC_TRANSIT = Beta('ASC_TRANSIT',betas['ASC_TRANSIT'],None,None,0)
B_TIME = Beta('B_TIME',betas['B_TIME'],None,None,0)
B_TIME_BC = Beta('B_TIME_BC',betas['B_TIME_BC'],None,None,0)
B_COST = Beta('B_COST',betas['B_COST'],None,None,0)
B_DIST = Beta('B_DIST',betas['B_DIST'],None,None,0)

prob1_nw = models.logit(V,av,1)
prob2_nw = models.logit(V,av,2)
prob3_nw = models.logit(V,av,3)
prob4_nw = models.logit(V,av,4)

We can then calculate the set of elasticities. We can then complete the process using the .simulate() function. This will calculate the probabilities and elasticities for each observation. The Pandas .describe() function is useful to print the statistics for each of these calculated values (i.e. mean, minimum, maximum, standard deviation, percentiles).

In [ ]:
logitelas1_nw = WALK_AV * (1.0 - prob1_nw) * DIST * B_DIST
logitelas2_nw = BIKE_AV * (1.0 - prob2_nw) * DIST * B_DIST
logitelas3_nw = (1.0 - prob3_nw) * AUTO_TT * B_TIME
logitelas4_nw = (1.0 - prob4_nw) * TRANSIT_TT_BC * B_TIME_BC

simulate_nw = {'Prob. walk': prob1_nw,
            'Prob. bike': prob2_nw,
            'Prob. auto':prob3_nw,
            'Prob. transit':prob4_nw,
            'logit elas. 1':logitelas1_nw,
            'logit elas. 2':logitelas2_nw,
            'logit elas. 3':logitelas3_nw,
            'logit elas. 4':logitelas4_nw}

parameter_values = {
    'ASC_BIKE': pandasResults_nw.loc['ASC_BIKE', 'Value'],
    'ASC_TRANSIT': pandasResults_nw.loc['ASC_TRANSIT', 'Value'],
    'ASC_WALK': pandasResults_nw.loc['ASC_WALK', 'Value'],
    'B_COST': pandasResults_nw.loc['B_COST', 'Value'],
    'B_DIST': pandasResults_nw.loc['B_DIST', 'Value'],
    'B_TIME': pandasResults_nw.loc['B_TIME', 'Value'],
    'B_TIME_BC': pandasResults_nw.loc['B_TIME_BC', 'Value'] }

biogeme_nw  = bio.BIOGEME(database,simulate_nw)
biogeme_nw.modelName = "01logit_simul_nw"
sim_results_nw = biogeme_nw.simulate(theBetaValues=parameter_values)
print("Non-work results=\n",sim_results_nw.describe())

The chosen alternative [`2.0`] is not available for the following observations (rownumber[choice]): 0[2.0]-2[2.0]-62[2.0]-131[2.0]-132[2.0]-133[2.0]-134[2.0]-151[2.0]-156[2.0]-157[2.0]-179[2.0]-180[2....
The chosen alternative [`1.0`] is not available for the following observations (rownumber[choice]): 759[1.0]-975[1.0]-1251[1.0]-1405[1.0]-1406[1.0]-1878[1.0]-1879[1.0]-1880[1.0]-1881[1.0]-2009[1.0]-20...
The chosen alternative [`2.0`] is not available for the following observations (rownumber[choice]): 0[2.0]-2[2.0]-62[2.0]-131[2.0]-132[2.0]-133[2.0]-134[2.0]-151[2.0]-156[2.0]-157[2.0]-179[2.0]-180[2....


Non-work results=
          Prob. walk    Prob. bike    Prob. auto  Prob. transit  logit elas. 1  \
count  12500.000000  12500.000000  12500.000000   1.250000e+04   12500.000000   
mean       0.153120      0.011280      0.817360   1.823977e-02      -6.731711   
std        0.168670      0.012426      0.176500   1.358506e-02      14.189699   
min        0.000000      0.000000      0.000042   8.785730e-07    -197.294961   
25%        0.002183      0.000161      0.674123   1.359386e-02      -6.374614   
50%        0.078344      0.005772      0.896439   1.834815e-02      -2.482661   
75%        0.290058      0.021368      0.972936   2.090225e-02      -0.766167   
max        0.931345      0.068612      0.980577   7.674145e-01       0.666965   

       logit elas. 2  logit elas. 3  logit elas. 4  
count   12500.000000   12500.000000   12500.000000  
mean       -3.717315      -0.016373      -0.981119  
std         4.349744       0.075635       0.013467  
min       -22.228442      -4.041094    

In [ ]:
# We calculate the aggregate elasticities

# First the denominator of the aggregate elasticity expression.
denominator_walk_nw = sim_results_nw['Prob. walk'].sum()
denominator_bike_nw = sim_results_nw['Prob. bike'].sum()
denominator_auto_nw = sim_results_nw['Prob. auto'].sum()
denominator_transit_nw = sim_results_nw['Prob. transit'].sum()

# And now the aggregate elasticities themselves.
direct_elas_term_walk_dist_nw = (
    sim_results_nw['Prob. walk']
    * sim_results_nw['logit elas. 1']
    / denominator_walk_nw
).sum()
print(
    f'Aggregate direct point elasticity of walk wrt distance for non-work: '
    f'{direct_elas_term_walk_dist_nw:.3g}'
)

direct_elas_term_bike_dist_nw = (
    sim_results_nw['Prob. bike']
    * sim_results_nw['logit elas. 2']
    / denominator_bike_nw
).sum()
print(
    f'Aggregate direct point elasticity of bike wrt distance for non-work: '
    f'{direct_elas_term_bike_dist_nw:.3g}'
)

direct_elas_term_auto_time_nw = (
    sim_results_nw['Prob. auto']
    * sim_results_nw['logit elas. 3']
    / denominator_auto_nw
).sum()
print(
    f'Aggregate direct point elasticity of auto wrt time for non-work: '
    f'{direct_elas_term_auto_time_nw:.3g}'
)

direct_elas_term_transit_time_nw = (
    sim_results_nw['Prob. transit']
    * sim_results_nw['logit elas. 4']
    / denominator_transit_nw
).sum()
print(
    f'Aggregate direct point elasticity of transit wrt time for non-work: '
    f'{direct_elas_term_transit_time_nw:.3g}'
)

Aggregate direct point elasticity of walk wrt distance for non-work: -0.764
Aggregate direct point elasticity of bike wrt distance for non-work: -0.946
Aggregate direct point elasticity of auto wrt time for non-work: -0.0127
Aggregate direct point elasticity of transit wrt time for non-work: -0.971


# Home-Based Work Trip Model

In [ ]:
hbw_filt = (my_trips.trippurp=="HBW")
hbw_trips = my_trips.loc[hbw_filt].select_dtypes(exclude=['object'])
hbw_trips.fillna(-1, inplace=True)
hbw_trips["walk_av"] = (hbw_trips.trpmiles   <=  hbw_trips[hbw_trips.ch_mode==1].trpmiles.max()).astype(int)
hbw_trips["bike_av"] = (hbw_trips.trpmiles   <=  hbw_trips[hbw_trips.ch_mode==2].trpmiles.max()).astype(int)
hbw_trips["auto_av"] = 1
hbw_trips["transit_av"] = 1

We don't want to include a record if it has a negative value in a critical column. We'll filter the dataframe for those important columns.

In [ ]:
# Update based on if you add more columns to models that potentially contain negative numbers
check_cols = ["auto_tt","auto_cost", "transit_tt", "transit_acct", "transit_egrt", "transit_waitt", "trpmiles", "ch_mode"]
hbw_trips = hbw_trips[hbw_trips.loc[:,check_cols].sum(axis=1)>-1]

In [ ]:
# Need to import data to the Biogeme database structure
database = db.Database("my_hbw_trips",hbw_trips)
database.data.head()

# Define Biogeme variables
AUTO_TT = Variable("auto_tt")
AUTO_COST = Variable("auto_cost")
TRANSIT_TT = Variable("transit_tt")
DIST = Variable("trpmiles")
CHOICE = Variable("ch_mode")
WALK_AV = Variable("walk_av")
BIKE_AV = Variable("bike_av")
AUTO_AV = Variable("auto_av")
TRANSIT_AV = Variable("transit_av")
TRANSIT_ACCT = Variable("transit_acct")
TRANSIT_EGRT = Variable("transit_egrt")
TRANSIT_WAITT = Variable("transit_waitt")

We can then define the names and starting values for the model parameters. There are 5 parameters defined for the base Swiss Metro model. Biogeme uses a constrained optimization to reduce the search space. The Beta function is called as follows:
```python
Beta(parameter name AS str, initial value AS int, lower bound AS int, upper bound AS int, fixed AS int)
```
If there are no bounds on the parameter search space, the use may enter: ```None```. A value of ```0``` for the ```fixed``` parameter will allow Biogeme to estimate the parameter. In this case, we have only fixed the ASC for the swiss metro mode, to maintain model indentification.

In [ ]:
ASC_WALK = Beta('ASC_WALK',0,None,None,0)
ASC_BIKE = Beta('ASC_BIKE',0,None,None,0)
ASC_TRANSIT = Beta('ASC_TRANSIT',0,None,None,0)
B_TIME = Beta('B_TIME',0,None,None,0)
B_COST = Beta('B_COST',0,None,None,0)
B_DIST = Beta('B_DIST',0,None,None,0)

In [ ]:
print (B_TIME)
print (B_COST)
print (B_DIST)

B_TIME(init=0)
B_COST(init=0)
B_DIST(init=0)


In [ ]:
B_COST_BCW = Beta('B_COST_BCW',0,None,None,0)
AUTO_COST_BCW = models.boxcox(AUTO_COST,B_COST_BCW)

In [ ]:
B_TIME_BCW = Beta('B_TIME_BCW',0,None,None,0)
AUTO_TT_BCW = models.boxcox(AUTO_TT,B_TIME_BCW)
TRANSIT_TT_BCW = models.boxcox(TRANSIT_TT, B_TIME_BCW)

The next step is to define the utility function for each alternative mode.

In [ ]:
V1 = ASC_WALK + \
     B_DIST * DIST
V2 = ASC_BIKE + \
     B_DIST * DIST
V3 = B_TIME * AUTO_TT_BCW + \
     B_COST * AUTO_COST_BCW
V4 = ASC_TRANSIT + \
     B_TIME * TRANSIT_TT_BCW + B_TIME * TRANSIT_WAITT

Biogeme has a variety of built-in discrete choice models, which the user can leverage. However, it also allows the user to define their own likelihood function if it is not available in the built-in model library. In this case, we use the built-in ```bioLogLogit``` function to estimate the model.

In [ ]:
# Associate utility functions with the numbering of alternatives
V = {1: V1,
     2: V2,
     3: V3,
     4: V4}

# Associate the availability conditions with the alternatives

av = {1: WALK_AV,
      2: BIKE_AV,
      3: AUTO_AV,
      4: TRANSIT_AV}

logprob_w = models.loglogit(V,av,CHOICE)
biogeme_w  = bio.BIOGEME(database,logprob_w)
biogeme_w.modelName = "01logit"
rdistesults_w = biogeme_w.estimate()

The results of the model estimation can then be printed to the terminal.

In [ ]:
# Get the results in a pandas table
pandasResults_w = rdistesults_w.getEstimatedParameters()
print(pandasResults_w)


                 Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_BIKE    -11.588776      0.407823   -28.416185  0.000000e+00
ASC_TRANSIT  -6.297883      1.564737    -4.024882  5.700398e-05
ASC_WALK    -10.946939      0.425472   -25.728943  0.000000e+00
B_COST      -24.173265      0.184635  -130.924390  0.000000e+00
B_COST_BCW   -2.478654      0.001983 -1249.927871  0.000000e+00
B_DIST       -1.186812      0.185913    -6.383681  1.728810e-10
B_TIME       -0.680973      0.171994    -3.959284  7.517491e-05
B_TIME_BCW   -0.211038      0.071543    -2.949791  3.179886e-03


**NOTE:** The above model gives appropriate results for the sample CBSA! This may not be true for your dataset. DO NOT submit results with positive cost, distance, or time parameters.

Biogeme also includes functionalities to compute elasticties for model variables. The elasticity can be calculated directly by the user, but Biogeme also includes functions for calculating derivatives for the user.

We have to redefine the model for it to run in a Jupyter notebook, so there is some repetition in the following code. Note: the parameter definitions have been slightly altered to use the calculated betas. These are stored in a dictionary and the values can be accessed through reference to the appropriate dictionary keys.

In [ ]:
# Get the beta parameters estimated in previous step
betas = rdistesults_w.getBetaValues()

ASC_WALK = Beta('ASC_WALK',betas['ASC_WALK'],None,None,0)
ASC_BIKE = Beta('ASC_BIKE',betas['ASC_BIKE'],None,None,0)
ASC_TRANSIT = Beta('ASC_TRANSIT',betas['ASC_TRANSIT'],None,None,0)
B_TIME = Beta('B_TIME',betas['B_TIME'],None,None,0)
B_COST = Beta('B_COST',betas['B_COST'],None,None,0)
B_DIST = Beta('B_DIST',betas['B_DIST'],None,None,0)
B_TIME_BCW = Beta('B_TIME_BCW',betas['B_TIME_BCW'],None,None,0)
B_COST_BCW = Beta('B_COST_BCW',betas['B_COST_BCW'],None,None,0)

prob1_w = models.logit(V,av,1)
prob2_w = models.logit(V,av,2)
prob3_w = models.logit(V,av,3)
prob4_w = models.logit(V,av,4)

We can then calculate the set of elasticities. We can then complete the process using the .simulate() function. This will calculate the probabilities and elasticities for each observation. The Pandas .describe() function is useful to print the statistics for each of these calculated values (i.e. mean, minimum, maximum, standard deviation, percentiles).

In [ ]:
logitelas1_w = WALK_AV * (1.0 - prob1_w) * DIST * B_DIST
logitelas2_w = BIKE_AV * (1.0 - prob2_w) * DIST * B_DIST
logitelas3_w = (1.0 - prob3_w) * AUTO_TT_BCW * B_TIME_BCW
logitelas4_w = (1.0 - prob4_w) * TRANSIT_TT_BCW * B_TIME_BCW

simulate_w = {'Prob. walk': prob1_w,
            'Prob. bike': prob2_w,
            'Prob. auto':prob3_w,
            'Prob. transit':prob4_w,
            'logit elas. 1':logitelas1_w,
            'logit elas. 2':logitelas2_w,
            'logit elas. 3':logitelas3_w,
            'logit elas. 4':logitelas4_w}

parameter_values = {
    'ASC_BIKE': pandasResults_w.loc['ASC_BIKE', 'Value'],
    'ASC_TRANSIT': pandasResults_w.loc['ASC_TRANSIT', 'Value'],
    'ASC_WALK': pandasResults_w.loc['ASC_WALK', 'Value'],
    'B_COST': pandasResults_w.loc['B_COST', 'Value'],
    'B_DIST': pandasResults_w.loc['B_DIST', 'Value'],
    'B_TIME': pandasResults_w.loc['B_TIME', 'Value'],
    'B_COST_BCW': pandasResults_w.loc['B_COST_BCW', 'Value'],
    'B_TIME_BCW': pandasResults_w.loc['B_TIME_BCW', 'Value'],
}

biogeme_w  = bio.BIOGEME(database,simulate_w)
biogeme_w.modelName = "01logit_simul"
sim_results_w = biogeme_w.simulate(theBetaValues=parameter_values)
print("Results=",sim_results_w.describe())

The chosen alternative [`2.0`] is not available for the following observations (rownumber[choice]): 0[2.0]-1[2.0]-4[2.0]-5[2.0]-6[2.0]-9[2.0]-10[2.0]-11[2.0]-13[2.0]-15[2.0]-16[2.0]-18[2.0]-19[2.0]-23...
The chosen alternative [`1.0`] is not available for the following observations (rownumber[choice]): 0[1.0]-1[1.0]-4[1.0]-5[1.0]-6[1.0]-9[1.0]-10[1.0]-11[1.0]-13[1.0]-15[1.0]-16[1.0]-18[1.0]-19[1.0]-24...
The chosen alternative [`2.0`] is not available for the following observations (rownumber[choice]): 0[2.0]-1[2.0]-4[2.0]-5[2.0]-6[2.0]-9[2.0]-10[2.0]-11[2.0]-13[2.0]-15[2.0]-16[2.0]-18[2.0]-19[2.0]-23...


Results=         Prob. walk   Prob. bike   Prob. auto  Prob. transit  logit elas. 1  \
count  2803.000000  2803.000000  2803.000000   2.803000e+03    2803.000000   
mean      0.027114     0.014271     0.919372   3.924393e-02      -2.725433   
std       0.067783     0.035676     0.124463   7.794184e-02       3.505564   
min       0.000000     0.000000     0.035374   2.298514e-11     -11.637763   
25%       0.000000     0.000000     0.946262   2.427901e-02      -5.174057   
50%       0.000022     0.000000     0.965016   3.033007e-02      -0.423044   
75%       0.009444     0.004971     0.971608   3.462688e-02       0.000000   
max       0.412257     0.216981     1.000000   9.631881e-01       0.000000   

       logit elas. 2  logit elas. 3  logit elas. 4  
count    2803.000000    2803.000000    2803.000000  
mean       -2.471384      -0.035339      -0.549832  
std         3.256771       0.054335       0.054640  
min       -10.981461      -0.476528      -0.655451  
25%        -4.688466   

In [ ]:
# We calculate the aggregate elasticities

# First the denominator of the aggregate elasticity expression.
denominator_walk_w = sim_results_w['Prob. walk'].sum()
denominator_bike_w = sim_results_w['Prob. bike'].sum()
denominator_auto_w = sim_results_w['Prob. auto'].sum()
denominator_transit_w = sim_results_w['Prob. transit'].sum()

# And now the aggregate elasticities themselves.
direct_elas_term_walk_dist_w = (
    sim_results_w['Prob. walk']
    * sim_results_w['logit elas. 1']
    / denominator_walk_w
).sum()
print(
    f'Aggregate direct point elasticity of walk wrt distance: '
    f'{direct_elas_term_walk_dist_w:.3g}'
)

direct_elas_term_bike_dist_w = (
    sim_results_w['Prob. bike']
    * sim_results_w['logit elas. 2']
    / denominator_bike_w
).sum()
print(
    f'Aggregate direct point elasticity of bike wrt distance: '
    f'{direct_elas_term_bike_dist_w:.3g}'
)

direct_elas_term_auto_time_w = (
    sim_results_w['Prob. auto']
    * sim_results_w['logit elas. 3']
    / denominator_auto_w
).sum()
print(
    f'Aggregate direct point elasticity of auto wrt time: '
    f'{direct_elas_term_auto_time_w:.3g}'
)

direct_elas_term_transit_time_w = (
    sim_results_w['Prob. transit']
    * sim_results_w['logit elas. 4']
    / denominator_transit_w
).sum()
print(
    f'Aggregate direct point elasticity of transit wrt time: '
    f'{direct_elas_term_transit_time_w:.3g}'
)

Aggregate direct point elasticity of walk wrt distance: -1.36
Aggregate direct point elasticity of bike wrt distance: -1.45
Aggregate direct point elasticity of auto wrt time: -0.0282
Aggregate direct point elasticity of transit wrt time: -0.46


# Supplemental notes
Biogeme provides many other functions for viewing results: <br> getGeneralStatistics() <br> getEstimatedParameters() <br> getCorrelationResults() <br> getBetaValues() <br> getRobustVarCovar() <br> getBetasForSensitivityAnalysis()

The manuals are also quite helpful: <br>
http://biogeme.epfl.ch/documents.html

# **To Do List:**
1. Develop a multinomial logit (MNL) mode choice model for home-based non-work trips. Some experimentation will likely be required to find the “best” model that can be obtained given the available data and the model type chosen. Present and discuss the results of your analysis in terms of the statistical significance of the parameter estimates, the goodness-of-fit of the model, and the rationale for the variables selected for inclusion in the model. Your discussion should also include consideration of elasticity estimates.


2. Similarly develop an MNL mode choice model for home-based work trips. Compare your results with those obtained for the home-based non-work trip model with respect to explanatory variables used, parameter statistical significance, and model goodness-of-fit. Discuss likely reasons for these differences (if any). Your discussion should also include consideration of elasticity estimates.

**Home Based Non-Work Trips Model**

-Discuss model chosen

-stat significance of model

-gof for model

-variables included
For this model, we decided to not include any additonal variables beyond the variables provided for our model. From the available list of variables, we utlized four utility equations to analyze our data. The first equation included the alternative specific constant of walking multiplied by the distance of the trip and its respective beta parameter. The second equation included the alternative specific constant of biking multiplied by the distance of the trip and its respective beta parameter. The third equation included both the cost of a car, the travel time in the car, and the respective beta parameters for each. The fourth equation included the alternative specific constant of taking public transit, the travel time of the transit method, and the beta parameter associated with both.

-elasticity estimates

**Home Based Work Trips Model**

-Discuss model chosen
-stat significance of model
-gof for model

For this model,  we utlized four utility equations to analyze our data. The first equation included the alternative specific constant of walking multiplied by the distance of the trip and its respective beta parameter. The second equation included the alternative specific constant of biking multiplied by the distance of the trip and its respective beta parameter. The third equation included both the cost of a car, the travel time in the car, and the respective beta parameters for each. The fourth equation included both the cost of transit, the travel time of the transit method, and the beta parameter associated with both.


-elasticity estimates

**Beta Parameters**

For our home-based non-work trip parameters, we began with a negative distance parameter and a positve cost and time parameter. Considering what these parameters represent, this is to be expected. Cost typically is a positive parameter because people tend to be okay with paying more if there are luxury qualities associated with the most expensive cost. Similar to cost, a positive time parameter represents trip makers choosing a longer trip if there are luxuries associated with the trip. However, a negative parameter is expected for all three parameters. To adjust for this a boxcox transformation is used. After using this transformation time parameters, this new parameter was applied to the Transit travel time variable. With this adjustment, all the beta parameters were calculated to be negative.

For the home-based work trip parameters, we again begn with a negative distance parameter and a positive cost and time parameter. The same method previously described was used to correct these factors to be negative. For this model, we had to use the boxcox transformation for both the cost and the time parameter. The adjusted cost parameter was applied to the auto cost variable and the adjusted time parameter was applied to the transit travel time and the transit wait time variable.

**Comparision of Model**
